[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JoshWilde/JOSS_Reviewer_Matcher/blob/main/Idea%202/JOSS_Reviewer_Idea_2.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string

In [2]:
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.probability import FreqDist
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
PAPER_OF_INTEREST_FNAME  =  glob.glob('/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/*/*/*.pdf')

In [5]:
# https://zenodo.org/record/3631674#.YeglfP7P2Uk
df = pd.read_csv('/Volumes/Seagate Backup Plus Drive/JOSS Project/wiki_tfidf_terms.csv')
df_reviewers = pd.read_csv('../Data/JOSS Table Test.csv')

In [6]:
def Get_Lemma_Words(POI_PDF):
  text = str(POI_PDF)
  text2 = text.split()
  words_no_punc = []

  for w in text2:
    if w.isalpha():
      words_no_punc.append(w.lower())
  from nltk.corpus import stopwords
  stopwords = stopwords.words('english')  
  clean_words = []
  for w in words_no_punc:
    if w not in stopwords:
      clean_words.append(w)
  clean_words_arr = ''
  for i in range(len(clean_words)):
    clean_words_arr = clean_words_arr + ' ' + str(clean_words[i])

  string_for_lemmatizing = clean_words_arr
  lemmatizer = WordNetLemmatizer()
  words_2 = word_tokenize(string_for_lemmatizing)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words_2]

  lemmatized_words_arr = ''
  for i in range(len(lemmatized_words)):
    lemmatized_words_arr = lemmatized_words_arr + ' ' + str(lemmatized_words[i])
  words = word_tokenize(lemmatized_words_arr)

  return words

In [7]:
def Get_Top_Words_tf_idf(Paper_interest, df=df, df_reviewers=df_reviewers, num_suggestions=5, num_top20=20):
    POI_PDF = [extract_text(Paper_interest)]
    text = str(POI_PDF)
    words = Get_Lemma_Words(POI_PDF)
    #print(len(words))
    fdist = FreqDist(words)
    X = np.array(fdist.most_common())
    tf_idf_arr_names, tf_idf_arr_floats = determine_wiki_td_idf(X, df=df)
    #print('determined wiki')
    num_arr = np.array(tf_idf_arr_floats)
    tf_idf_arr_names_arr = np.array(tf_idf_arr_names)
    top20_tf_idf = tf_idf_arr_names_arr[np.argsort(num_arr)[:num_top20]]
    return top20_tf_idf

In [8]:
def Compare_topics(top20, df_reviewers):
  length = df_reviewers.shape[0] - 1
  match_arr = np.zeros(length)
  for i in range(length):
    if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].str.lower().values[1+i]) == False:
      t = df_reviewers['Domains/topic areas you are comfortable reviewing'].str.lower().values[1+i]
      #print(i)
      uniarr = Split_columns(t)
      for j in range(len(uniarr)):
        for k in range(len(top20)):
          if uniarr[j] == top20[k]:
            match_arr[i] = match_arr[i] + 1
  return match_arr

In [9]:
def Split_columns(t):
  txt = " ".join("".join([" " if ch in string.punctuation else ch for ch in t]).split())
  sol1 = np.char.split(txt, ' ')
  txt_arr  = array_of_lists_to_array(sol1)
  uniarr = np.unique(txt_arr)
  return uniarr

In [10]:
 def array_of_lists_to_array(arr):
    return np.apply_along_axis(lambda a: np.array(a[0]), -1, arr[..., None])

In [11]:
def determine_wiki_td_idf(x, df=df):
    tf_idf_arr_names = []
    tf_idf_arr_floats = []
    for i in range(len(x)):
        if df[df['token'] ==x[i][0]].frequency.empty == False:
            wiki_tf = df[df['token'] ==x[i][0]].frequency.values[0]
            doc_tf = int(x[i][1])
            tf_idf = np.log(wiki_tf/doc_tf)
            tf_idf_arr_names.append(x[i][0])
            tf_idf_arr_floats.append(tf_idf)
    return tf_idf_arr_names, tf_idf_arr_floats

In [12]:
def summatation_bot(all_usernames, all_domains, all_num_matched_words, all_matched_words):
  length = len(all_usernames)
  message = 'Hello. \nI have found ' + str(length) + ' possible reviewers for this paper.' +'\n\n'
  for i in range(length):
    ps = 'I believe ' + all_usernames[i] + ' will make a good reviewer for this paper because they have matched ' + str(int(all_num_matched_words[i])) +  ' words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ' + str(all_matched_words[i]) +'.\nFrom their topics domain: ' + str(all_domains[i].replace('\n', ', ')) +'.\n'
    message = message + ps + '\n'
  print(message)

In [13]:
Q = 340
print(PAPER_OF_INTEREST_FNAME[Q])

/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.00775/10.21105.joss.00775.pdf


In [14]:
top20_tf_idf = Get_Top_Words_tf_idf(PAPER_OF_INTEREST_FNAME[Q])

In [15]:
top20_tf_idf

array(['helpme', 'uservoice', 'helper', 'serf', 'endpoint', 'github',
       'headless', 'ticketing', 'interrupt', 'intuitive', 'needing',
       'user', 'click', 'repository', 'attribution', 'navigate', 'ample',
       'manually', 'software', 'tool'], dtype='<U13')

In [16]:
def GetReviewer_Vectors(df_reviewers=df_reviewers):
    reviewer_vectors = np.zeros(((df_reviewers.shape[0]-1),300))
    for i in range(df_reviewers.shape[0]-1):
        #if i%10 == 0:
        #    print(i)
        if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i]) == False:
            review_text = df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i].lower()
            review_text = review_text.replace('-\\n','')
            review_text = review_text.replace('\\n',' ')
            review_text = review_text.replace('\n', ' ')
        
            review_arr = []
            for token in model(review_text):
                if token.is_alpha == True:
                    if token.is_stop == False:
                        review_arr.append(str(token.lemma_).lower())
            review_arr = np.array(review_arr)
        
            review_str = ''
            for j in np.unique(review_arr):
                review_str = review_str + j +' '
        
        #print(model(review_str).vector.shape)
        #print(reviewer_vectors.shape)
            reviewer_vectors[i] = model(review_str).vector
    
    return reviewer_vectors

In [17]:
def GetCosineSims(doc_vec, review_vec, df_reviewers=df_reviewers):
    all_usernames = []
    all_domains = []
    all_cosine_sims = []
    for j in range(len(review_vec)):
        if pd.isna(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing']) == False:
            all_cosine_sims.append(cosine_similarity(np.array([doc_vec]), np.array([review_vec[j]]))[0,0])
            all_domains.append(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing'].lower())
            all_usernames.append(df_reviewers.iloc[j+1].username)
    all_usernames= np.array(all_usernames)
    all_domains= np.array(all_domains)
    all_cosine_sims= np.array(all_cosine_sims)
    
    return all_usernames, all_domains, all_cosine_sims

In [18]:
model = spacy.load('en_core_web_lg')

In [19]:
reviewer_vectors = GetReviewer_Vectors()

In [20]:
top20_tf_idf

array(['helpme', 'uservoice', 'helper', 'serf', 'endpoint', 'github',
       'headless', 'ticketing', 'interrupt', 'intuitive', 'needing',
       'user', 'click', 'repository', 'attribution', 'navigate', 'ample',
       'manually', 'software', 'tool'], dtype='<U13')

In [21]:
doc_top20= ''
for i in top20_tf_idf:
    doc_top20 = doc_top20 + i +' '

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
all_usernames2, all_domains2, all_cosine_sims2 = GetCosineSims(model(doc_top20).vector, reviewer_vectors)

In [25]:
def TopReviewers(number=5, all_usernames=all_usernames2, all_domains=all_domains2, all_cosine_sims=all_cosine_sims2):
    message = 'Hello.\n I have found ' +str(number) + ' possible reviewers for this paper.'+ '\n\n'
    for J in range(number):
        index = np.argsort(all_cosine_sims)[-1-J]
        #print(index)
        ps = 'I believe '+ colored(str(all_usernames[index]), 'green') + ' will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of ' + colored(str(all_cosine_sims[index])[:6], 'blue') + '. This reviewers domain interests are ' + colored(str(all_domains[index].replace('\n', ',')), 'red')
        message = message + ps + '\n\n'
    print(message)    

In [27]:
from termcolor import colored
import warnings

In [28]:
TopReviewers()

Hello.
 I have found 5 possible reviewers for this paper.

I believe GaurangTandon will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.7602. This reviewers domain interests are products that build on general web technologies (web servers/scrapers/frontend/backend), , userscripts/chrome extensions, , performance oriented applications (fast python/c++/rust, flamegraphs, perf stat)

I believe abhishekvp will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.7566. This reviewers domain interests are web dev tools, web browser, remote sensing tools, geospatial , gis, semantic web, knowledge representation, ontology, iot, hpc, embedded hpc

I believe jarrah42 will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.7212. This reviewers domain interests are high performance/scientific computing, operating systems, 